In [2]:
import pandas as pd
import numpy as np
import re

In [6]:
import os
path = "./fsdata"
file_list = os.listdir(path)
file_list

['3S.xlsx',
 'AJ네트웍스.xlsx',
 'AK홀딩스.xlsx',
 'BGF.xlsx',
 'BGF리테일.xlsx',
 'BNGT.xlsx',
 'BYC.xlsx',
 'CJ.xlsx',
 'CJ씨푸드.xlsx',
 'CMG제약.xlsx',
 'CS홀딩스.xlsx',
 'DB.xlsx',
 'DB하이텍.xlsx',
 'DMS.xlsx',
 'DRB동일.xlsx',
 'DSR.xlsx',
 'E1.xlsx',
 'ESR켄달스퀘어리츠.xlsx',
 'ES큐브.xlsx',
 'GH신소재.xlsx',
 'GS.xlsx',
 'GS건설.xlsx',
 'GS글로벌.xlsx',
 'GS리테일.xlsx',
 'HDC.xlsx',
 'HDC현대산업개발.xlsx',
 'HJ중공업.xlsx',
 'JTC.xlsx',
 'JW생명과학.xlsx',
 'JW신약.xlsx',
 'JW중외제약.xlsx',
 'JW홀딩스.xlsx',
 'JYP Ent..xlsx',
 'KBI메탈.xlsx',
 'KB오토시스.xlsx',
 'KCC건설.xlsx',
 'KC그린홀딩스.xlsx',
 'KC코트렐.xlsx',
 'KD.xlsx',
 'KEC.xlsx',
 'KG ETS.xlsx',
 'KG동부제철.xlsx',
 'KG모빌리언스.xlsx',
 'KG이니시스.xlsx',
 'KG케미칼.xlsx',
 'KH E&T.xlsx',
 'KH 일렉트론.xlsx',
 'KH 필룩스.xlsx',
 'KH바텍.xlsx',
 'KISCO홀딩스.xlsx',
 'KMH.xlsx',
 'KMH하이텍.xlsx',
 'KNN.xlsx',
 'KPX케미칼.xlsx',
 'KPX홀딩스.xlsx',
 'KR모터스.xlsx',
 'KSS해운.xlsx',
 'KTcs.xlsx',
 'LF.xlsx',
 'LG.xlsx',
 'LG디스플레이.xlsx',
 'LG생활건강.xlsx',
 'LG에너지솔루션.xlsx',
 'LG유플러스.xlsx',
 'LG이노텍.xlsx',
 'LG전자.xlsx',
 'LG화학.xlsx',
 'LI

In [99]:
con_df_li = []
count = 0
error_li = []
for i, f in enumerate(file_list):
    df = pd.read_excel(f"./fsdata/{f}", sheet_name=None)
    
    if df['Data_cis'].iloc[:, 1][0] == "label_ko":
        label_col = 1
    else:
        label_col = 2

    if 'Data_is' in df.keys():
        sheet_name = 'Data_is'
    else:
        sheet_name = 'Data_cis'
        
    # 자본금   
    df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col].str.contains('자본금').fillna(False)] 
    if len(df1) > 1:
        df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col] == '자본금']
    elif len(df1) == 0:
        df1 = df['Data_bs'][df['Data_bs'].iloc[:, 1].str.contains('ifrs-full_IssuedCapital').fillna(False)] 
        if len(df1) == 0:
            df1 = df['Data_bs'][df['Data_bs'].iloc[:, 1].isin(['ifrs_EquityAttributableToOwnersOfParent']).fillna(False)] 
    # 자산총계
    df2 = df['Data_bs'][df['Data_bs'].iloc[:, label_col] == '자산총계']
    
    # 당기순이익
    df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('당기순이익').fillna(False)]
    if len(df3) > 1:
        df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].isin(['당기순이익(손실)', '당기순이익', '당기순손익', '당기순손실'])]
        if len(df3) == 0:
            df3 = df['Data_cf'][df['Data_cf'].iloc[:, label_col].isin(['당기순이익(손실)'])]
    elif len(df3) == 0:
        df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].isin(['당기순손익', '당기순손실', '당기의 순이익', '계속영업이익(손실)'])]
        if len(df3) == 0:
            df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('당기손익').fillna(False)]
    if len(df3) > 1:
        df3 = df3[df3.isna().sum(axis = 1) == df3.isna().sum(axis = 1).min()]
    
    # 영업활동 현금흐름
    regex = r'[1-9]?[A-z]?[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]?\s?영업\s?활동?([가-힣]+)?(\s+[가-힣]+\s+)?\s?(?!창출된)현금\s?흐름'
    df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].str.contains('ifrs-full_CashFlowsFromUsedInOperatingActivities').fillna(False)]                                                        
    if len(df4) == 0:
        df4 = df['Data_cf'][df['Data_cf'].iloc[:, label_col].str.contains(regex).fillna(False)]
        if len(df4) > 1:
            df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].isin(['ifrs_CashFlowsFromUsedInOperatingActivities']).fillna(False)]
    
    # 영업 이익
    df5 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('dart_OperatingIncomeLoss').fillna(False)]
    if len(df5) == 0:
        df5 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'영업\s?이익').fillna(False)]
        if len(df5) == 0:
            df5 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains(r'영업\s?이익').fillna(False)]
            if len(df5) == 0:
                sheet_name = 'Data_cis'
                df5 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'영업\s?이익').fillna(False)]
                
    # 매출액
   
    df6 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('ifrs-full_Revenue').fillna(False)] 
    if len(df6) == 0:
        df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['ifrs-full_Revenue', 'ifrs_Revenue'])]
        if len(df6) == 0:
            df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains('매출액').fillna(False)]
            if len(df6) == 0:
                 df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].isin(['매출'])]
        if len(df6) == 0:
            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('매출액').fillna(False)]
            if len(df6) > 1:
                df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin([r'Ⅰ.매출액.+'])]

        if len(df6) == 0:
            df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains('영업수익').fillna(False)]
            
            if len(df6) == 0: 
                df6 = (df[sheet_name][df[sheet_name]
                            .iloc[:, 1]
                            .isin(['entity01152470_udf_IS_2018102918183181_StatementOfComprehensiveIncomeAbstract'])] 
                      
                      )
    elif len(df6) > 1:
        df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['ifrs-full_Revenue', 'ifrs_Revenue'])]
        
    
#     if len(df6) == 0:
#         df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains("매출액").fillna(False)]
    
#     print(i,f, len(df6))
#     if len(df6) != 1:
#         error_li.append([i,f, len(df6)])
#         count += 1
    print(i, f, len(df1), len(df2), len(df3), len(df4), len(df5), len(df6))
    if len(df1) != 1 or len(df2) != 1 or len(df3) != 1 or len(df4) != 1 or len(df5) != 1 or len(df6) != 1:
        count += 1
        error_li.append([i, f])
        print('Error===============================================================')
        
#     if len(df1) != 1 or len(df2) != 1 or len(df3) != 1 or len(df4) != 1 or len(df5) != 1:
#         count += 1
#         error_li.append([i, f])
#         print('Error===============================================================')

print('='*200)
for i in error_li:
    print(i)
    
print(count)

0 3S.xlsx 1 1 1 1 1 1
1 AJ네트웍스.xlsx 1 1 1 1 1 1
2 AK홀딩스.xlsx 1 1 1 1 1 1
3 BGF.xlsx 1 1 1 1 1 1
4 BGF리테일.xlsx 1 1 1 1 1 1
5 BNGT.xlsx 1 1 1 1 1 1
6 BYC.xlsx 1 1 1 1 1 1
7 CJ.xlsx 1 1 1 1 1 1
8 CJ씨푸드.xlsx 1 1 1 1 1 1
9 CMG제약.xlsx 1 1 1 1 1 1


C:\Users\bitcamp\Anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


10 CS홀딩스.xlsx 1 1 1 1 1 1
11 DB.xlsx 1 1 1 1 1 1
12 DB하이텍.xlsx 1 1 1 1 1 1
13 DMS.xlsx 1 1 1 1 1 1
14 DRB동일.xlsx 1 1 1 1 1 1
15 DSR.xlsx 1 1 1 1 1 1
16 E1.xlsx 1 1 1 1 1 1
17 ESR켄달스퀘어리츠.xlsx 1 1 1 1 1 1
18 ES큐브.xlsx 1 1 1 1 1 1
19 GH신소재.xlsx 1 1 1 1 1 1
20 GS.xlsx 1 1 1 1 1 1
21 GS건설.xlsx 1 1 1 1 1 1
22 GS글로벌.xlsx 1 1 1 1 1 1
23 GS리테일.xlsx 1 1 1 1 1 1
24 HDC.xlsx 1 1 1 1 1 1
25 HDC현대산업개발.xlsx 1 1 1 1 1 1
26 HJ중공업.xlsx 1 1 1 1 1 1
27 JTC.xlsx 1 1 1 1 1 1
28 JW생명과학.xlsx 1 1 1 1 1 1
29 JW신약.xlsx 1 1 1 1 1 1
30 JW중외제약.xlsx 1 1 1 1 1 1
31 JW홀딩스.xlsx 1 1 1 1 1 1
32 JYP Ent..xlsx 1 1 1 1 1 1
33 KBI메탈.xlsx 1 1 1 1 1 1
34 KB오토시스.xlsx 1 1 1 1 1 1
35 KCC건설.xlsx 1 1 1 1 1 1
36 KC그린홀딩스.xlsx 1 1 1 1 1 1
37 KC코트렐.xlsx 1 1 1 1 1 1
38 KD.xlsx 1 1 1 1 1 1
39 KEC.xlsx 1 1 1 1 1 1
40 KG ETS.xlsx 1 1 1 1 1 1
41 KG동부제철.xlsx 1 1 1 1 1 1
42 KG모빌리언스.xlsx 1 1 1 1 1 1
43 KG이니시스.xlsx 1 1 1 1 1 1
44 KG케미칼.xlsx 1 1 1 1 1 1
45 KH E&T.xlsx 1 1 1 1 1 1
46 KH 일렉트론.xlsx 1 1 1 1 1 1
47 KH 필룩스.xlsx 1 1 1 1 1 1
48 KH바텍.xls

C:\Users\bitcamp\Anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


85 SK가스.xlsx 1 1 1 1 1 1
86 SK네트웍스.xlsx 1 1 1 1 1 1
87 SK디앤디.xlsx 1 1 1 1 1 1
88 SK리츠.xlsx 1 1 1 1 1 1
89 SK이노베이션.xlsx 1 1 1 1 1 1
90 SK케미칼.xlsx 1 1 1 1 1 1
91 SK텔레콤.xlsx 1 1 1 1 1 1
92 SNT모티브.xlsx 1 1 1 1 1 1
93 SNT에너지.xlsx 1 1 1 1 1 1
94 SNT중공업.xlsx 1 1 1 1 1 1
95 SNT홀딩스.xlsx 1 1 1 1 1 1
96 SPC삼립.xlsx 1 1 1 1 1 1
97 STX.xlsx 1 1 1 1 1 1
98 STX엔진.xlsx 1 1 1 1 1 1
99 STX중공업.xlsx 1 1 1 1 1 1
100 TCC스틸.xlsx 1 1 1 1 1 1
101 THE E&M.xlsx 1 1 1 1 1 1
102 THE MIDONG.xlsx 1 1 1 1 1 1
103 THQ.xlsx 1 1 0 1 1 1
Error===============================================================
104 TJ미디어.xlsx 1 1 1 1 1 1
105 TPC.xlsx 1 1 1 1 1 1
106 TS트릴리온.xlsx 1 1 1 1 1 1
107 TYM.xlsx 1 1 1 1 1 1
108 WI.xlsx 1 1 1 1 1 1
109 WISCOM.xlsx 1 1 1 1 1 1
110 가비아.xlsx 1 1 1 1 1 1
111 가온미디어.xlsx 1 1 1 1 1 1
112 가온전선.xlsx 1 1 1 1 1 1
113 감성코퍼레이션.xlsx 1 1 1 1 1 1
114 강남제비스코.xlsx 1 1 1 1 1 1
115 강스템바이오텍.xlsx 1 1 1 1 1 1
116 강원랜드.xlsx 1 1 1 1 1 1
117 강원에너지.xlsx 1 1 1 1 1 1
118 갤럭시아머니트리.xlsx 1 1 1 1 1 1
119 갤럭시아에스엠.xlsx 1 1

...

C:\Users\bitcamp\Anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


479 삼성생명.xlsx 1 1 2 1 1 1
Error===============================================================
480 삼성에스디에스.xlsx 1 1 1 1 1 1
481 삼성엔지니어링.xlsx 1 1 1 1 1 1
482 삼성전기.xlsx 1 1 1 1 1 1
483 삼성전자.xlsx 1 1 1 1 1 1
484 삼성제약.xlsx 1 1 1 1 1 1
485 삼성중공업.xlsx 1 1 1 1 1 1
486 삼성증권.xlsx 1 1 1 1 1 1
487 삼성출판사.xlsx 1 1 1 1 1 1
488 삼아알미늄.xlsx 1 1 1 1 1 1
489 삼아제약.xlsx 1 1 1 1 1 1
490 삼양사.xlsx 1 1 1 1 1 1
491 삼양식품.xlsx 1 1 1 1 1 1
492 삼양옵틱스.xlsx 1 1 1 1 1 1
493 삼양통상.xlsx 1 1 1 1 1 1
494 삼양홀딩스.xlsx 1 1 1 1 1 1
495 삼영무역.xlsx 1 1 1 1 1 1
496 삼영엠텍.xlsx 1 1 1 1 1 1
497 삼영이엔씨.xlsx 1 1 1 1 1 1
498 삼익악기.xlsx 1 1 1 1 1 1
499 삼일제약.xlsx 1 1 1 1 1 1
500 삼정펄프.xlsx 1 1 1 1 1 1
501 삼지전자.xlsx 1 1 1 1 1 1
502 삼진엘앤디.xlsx 1 1 1 1 1 1
503 삼천당제약.xlsx 1 1 1 1 1 1
504 삼천리.xlsx 1 1 1 1 1 1
505 삼천리자전거.xlsx 1 1 1 1 1 1
506 삼표시멘트.xlsx 1 1 1 1 1 1
507 삼호개발.xlsx 1 1 1 1 1 1
508 삼화네트웍스.xlsx 1 1 1 1 1 1
509 삼화왕관.xlsx 1 1 1 1 1 1
510 삼화전기.xlsx 1 1 1 1 1 1
511 상보.xlsx 1 1 1 1 1 1
512 상상인.xlsx 1 1 1 1 1 1
513 상상인인더스트리.xlsx 1 1 1 1 1 1
514

C:\Users\bitcamp\Anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


643 지니언스.xlsx 1 1 1 1 1 1
644 지더블유바이텍.xlsx 1 1 1 1 1 1
645 지씨셀.xlsx 1 1 1 1 1 1
646 지어소프트.xlsx 1 1 1 1 1 1
647 지엔씨에너지.xlsx 1 1 1 1 1 1
648 지엔코.xlsx 1 1 1 1 1 1
649 지역난방공사.xlsx 1 1 1 1 1 1
650 지투알.xlsx 1 1 1 1 1 1
651 지티지웰니스.xlsx 1 1 1 1 1 1
652 진도.xlsx 1 1 1 1 1 1
653 진로발효.xlsx 1 1 1 1 1 1
654 진매트릭스.xlsx 1 1 1 1 1 1
655 진성티이씨.xlsx 1 1 1 1 1 1
656 진양홀딩스.xlsx 1 1 1 1 1 1
657 진에어.xlsx 1 1 1 1 1 1
658 진원생명과학.xlsx 1 1 1 1 1 1
659 진코스텍.xlsx 1 1 1 1 1 1
660 질경이.xlsx 1 1 1 1 1 1
661 참존글로벌.xlsx 1 1 1 1 1 1
662 창해에탄올.xlsx 1 1 1 1 1 1
663 청담러닝.xlsx 1 1 1 1 1 1
664 체리부로.xlsx 1 1 1 1 1 1
665 체시스.xlsx 1 1 1 1 1 1
666 초록뱀미디어.xlsx 1 1 1 1 1 1
667 초록뱀컴퍼니.xlsx 1 1 1 1 1 1
668 칩스앤미디어.xlsx 1 1 1 1 1 1
669 카스.xlsx 1 1 1 1 1 1
670 카카오게임즈.xlsx 1 1 1 1 1 1
671 카페24.xlsx 1 1 1 1 1 1
672 캐리소프트.xlsx 1 1 1 1 1 1
673 캐스텍코리아.xlsx 1 1 1 1 1 1
674 캠시스.xlsx 1 1 1 1 1 1
675 커머스마이너.xlsx 1 1 1 1 1 1
676 컨버즈.xlsx 1 1 1 1 1 1
677 컬러레이.xlsx 1 1 1 1 1 1
678 컴투스.xlsx 1 1 1 1 1 1
679 컴투스홀딩스.xlsx 1 1 1 1 1 1
680 케어랩스.xlsx 1 1 1

936 한진중공업홀딩스.xlsx 1 1 1 1 1 1
937 한진칼.xlsx 1 1 1 1 1 1
938 한창바이오텍.xlsx 1 1 1 1 1 1
939 한컴MDS.xlsx 1 1 1 1 1 1
940 한컴위드.xlsx 1 1 1 1 1 1
941 한프.xlsx 1 1 1 1 1 1
942 한화.xlsx 1 1 1 1 1 1
943 한화솔루션.xlsx 1 1 1 1 1 1
944 한화시스템.xlsx 1 1 1 1 1 1
945 한화에어로스페이스.xlsx 1 1 1 1 1 1
946 해성디에스.xlsx 1 1 1 1 1 1
947 해성옵틱스.xlsx 1 1 1 1 1 1
948 해태제과식품.xlsx 1 1 1 1 1 1
949 핸즈코퍼레이션.xlsx 1 1 1 1 1 1
950 헝셩그룹.xlsx 0 1 1 1 1 1
Error===============================================================
951 헬릭스미스.xlsx 1 1 1 1 1 1
952 현대건설.xlsx 1 1 1 1 1 1
953 현대건설기계.xlsx 1 1 1 1 1 1
954 현대공업.xlsx 1 1 1 1 1 1
955 현대그린푸드.xlsx 1 1 1 1 1 1
956 현대글로비스.xlsx 1 1 1 1 1 1
957 현대두산인프라코어.xlsx 1 1 1 1 1 1
958 현대로템.xlsx 1 1 1 1 1 1
959 현대모비스.xlsx 1 1 1 1 1 1
960 현대미포조선.xlsx 1 1 1 1 1 1
961 현대바이오.xlsx 1 1 1 1 1 1
962 현대바이오랜드.xlsx 1 1 1 1 1 1
963 현대백화점.xlsx 1 1 1 1 1 1
964 현대비앤지스틸.xlsx 1 1 1 1 1 1
965 현대약품.xlsx 1 1 1 1 1 1
966 현대에너지솔루션.xlsx 1 1 1 1 1 1
967 현대에버다임.xlsx 1 1 1 1 1 1
968 현대에이치티.xlsx 1 1 1 1 1 1
969 현대오토에버.xlsx 1 1 1 1 1 

In [98]:
df = pd.read_excel(f"./fsdata/금비.xlsx", sheet_name=None)
df['Data_cis']

,Unnamed: 0,"[D431410] Statement of comprehensive income, by function of expense - Consolidated financial statements (Unit: KRW)",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,20201001-20210930,20191001-20200930,20181001-20190930,20171001-20180930,20161001-20170930,20151001-20160930,20141001-20150930,20131001-20140930,20121001-20130930,20111001-20120930,20101001-20110930
0,NaN,concept_id,label_ko,label_en,class0,class1,class2,class3,"('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)"
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,ifrs-full_Revenue,매출액,Revenue(Sales),포괄손익계산서 [abstract],매출액,NaN,NaN,228581174447,219258395356,201404467525,196995588750,185531451959,200543666584,189495055962,177506458713,175061513251,176226094545,182184367911
3,1.0,ifrs-full_CostOfSales,매출원가,Cost of sales,포괄손익계산서 [abstract],매출원가,NaN,NaN,185852942337,176401174451,157515543413,153041065132,142510119975,147780840196,145599763957,143669633512,140973206590,139613667311,140978606507
4,2.0,ifrs-full_GrossProfit,매출총이익,Gross profit,포괄손익계산서 [abstract],매출총이익,NaN,NaN,42728232110,42857220905,43888924112,43954523618,43021331984,52762826388,43895292005,33836825201,34088306661,36612427234,41205761404
5,3.0,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,Selling general administrative expenses,포괄손익계산서 [abstract],판매비와관리비,NaN,NaN,32831541580,36220229225,31458729555,31249004357,31344908336,32770848577,28614843619,28304234176,28009184086,30514281910,33072978540
6,4.0,dart_OperatingIncomeLoss,영업이익(손실),Operating income(loss),포괄손익계산서 [abstract],영업이익(손실),NaN,NaN,9896690530,6636991680,12430194557,12705519261,11676423648,19991977811,15280448386,5532591025,6079122575,6098145324,8132782864
7,5.0,entity00168401_udf_IS_2017122211361964_Stateme...,영업외손익,Non-operating profit and loss,포괄손익계산서 [abstract],영업외손익,NaN,NaN,-2547021080,-9764905180,1663015378,-86528509,-309791882,-2193932124,-4574733323,-1050354626,-1969080963,-991611995,137170116
8,6.0,ifrs-full_FinanceIncome,금융수익,Finance income,포괄손익계산서 [abstract],금융수익,NaN,NaN,1087431535,1197182998,1582402558,1177081853,1013052632,890272569,695128448,461277728,417054709,482690416,429859220
9,7.0,ifrs-full_FinanceCosts,금융비용,Finance costs,포괄손익계산서 [abstract],금융비용,NaN,NaN,2221838312,2488725911,1294144338,1270203714,1288059801,1621911276,1895213888,2003403034,2178153686,2775921080,2176279617
